In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# first: rotation diff
com_palm = [0,0,0]
com_wrist_link = [0,0,0]

# translation from link to palm
translation_wristlink_to_palm = com_palm - com_wrist_link
# TODO: turn into a translation MATRIX (rodrigues matrix?)

# next: rotation difference
# how_much_rotation: in radians. use np.radians to covert if necessary
# roll, pitch, yaw.
rotation_noise = R.from_euler('xyz', how_much_rotation)
translation_noise =  

# 


# combine all the transforms

translation_wristlink_to_palm 

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# first: rotation diff
com_palm = [0,0,0]
com_wrist_link = [0,0,0]

# translation from link to palm
translation_wristlink_to_palm = com_palm - com_wrist_link
# TODO: turn into a translation MATRIX (rodrigues matrix?)

# next: rotation difference, from wrist link to palm link
R.from_euler('xyz', [])  # difference between two quaternions?





# how_much_rotation: in radians. use np.radians to covert if necessary
# roll, pitch, yaw.
rotation_noise = R.from_euler('xyz', how_much_rotation)
translation_noise =  

# next: translation noise in local coordinates


# combine all the transforms

translation_wristlink_to_palm 

SyntaxError: invalid syntax (<ipython-input-2-dcb2fbe3586e>, line 22)

In [ ]:

def rotate_around_axis(vec, theta, axis, input_unit='deg'):
    """
    rotate around an arbitrary axis
    vec: xyz
    theta: single angle, but now around arbitrary axis
    axis: defined as a 3d vector direction, xyz
    """
    rotation_radians = theta

    if input_unit == 'deg':
        rotation_radians = np.radians(theta)

    rotation_axis = np.array(axis)  # type conversion... just in case
    rotation_vector = rotation_radians * rotation_axis
    rotation = R.from_rotvec(rotation_vector)
    rotated_vec = rotation.apply(vec)
    return rotated_vec

In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [7]:
position_vector_at_ground = np.array([0, 0.18, 0.0654])
normal_orientation_rotation = R.from_euler(seq='XYZ', angles=[-1.57, 0, -1.57], degrees=False)  # caps because instrinsic rotation

In [23]:
normal_orientation_rotation.as_euler(seq='XYZ')


default_orientation_rotation = R.from_euler(seq='XYZ', angles=[0, 0, 0], degrees=False)

In [27]:
default_orientation_rotation.as_matrix() * normal_orientation_rotation.as_matrix()

array([[ 7.96326711e-04,  0.00000000e+00,  0.00000000e+00],
       [-0.00000000e+00,  6.34136230e-07,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  7.96326711e-04]])

In [36]:
# (red, green, blue)

In [38]:
# THIS IS THE GOLDEN GOOSE!! actually does extrinsic rotation noise
bruh = np.matmul(default_orientation_rotation.as_matrix(), normal_orientation_rotation.as_matrix())
new_mujoco_euler = R.from_matrix(bruh).as_euler(seq='XYZ')

desired_rotation = R.from_rotvec([0, 0, np.pi/2])
# noisy_ori = np.matmul(bruh, desired_rotation.as_matrix())
noisy_ori = np.matmul(desired_rotation.as_matrix(), bruh)
new_new = R.from_matrix(noisy_ori).as_euler(seq='XYZ')
list(new_new)

[-3.9612757518625585e-13, -1.5699999999998484, 0.0007963267945001462]

In [5]:
# okay now we have pure extrinsic rotation noise. but now we have to preserve our palm...

# this is the part where we can pivot the arm around the sphere

def rotate_around_origin(vec, how_much_rotation):
    # vec: xyz
    # how_much_rotation: in radians. use np.radians to covert if necessary
    # roll, pitch, yaw.
    rotation = R.from_euler('xyz', how_much_rotation)

    rotated_vec = rotation.apply(vec)
    return rotated_vec

list(rotate_around_origin([0,0.18,0.0654], [0, 0, np.pi/2]))

[-0.18, 3.9968028886505634e-17, 0.0654]

In [7]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def rotate_around_origin(vec, how_much_rotation):
    # vec: xyz
    # how_much_rotation: in radians. use np.radians to covert if necessary
    # roll, pitch, yaw.
    rotation = R.from_euler('xyz', how_much_rotation)

    rotated_vec = rotation.apply(vec)
    return rotated_vec

def rotate_around_sphere_mucrappo(rotvec_noise, local_rotation_pos_shift=np.array([0,0,0]), default_orientation=np.array([0,0,0]), grasp_translation=np.array([0,0.18,0.0654]), grasp_orientation=np.array([-1.57, 0, -1.57]), post_rotation_pos_shift=np.array([0,0,0])):
    """
    rotates a point around a sphere
    
    rotvec_noise: rotation noise to be apply extrinsically, and to rotate the coordinates accordingly
    default_orientation: the default orientation. should stay at 0s
    grasp_translation: mujoco's euler value for whatever grasp position you're using
    grasp_orientation: mujoco's pos value for whatever grasp position you're using
    local_rotation_pos_shift: shifting the hand in local coordinates before the shift
    post_rotation_pos_shift: shifting the hand in global coordinates, after the inner shift...
    
    indep_rotation_noise: rotation noise that rotates without any consequence to th
    """
    # get rotation objects from starting grasp orientation, and the default orientation (should be 0 0 0)
    grasp_orientation_rotation = R.from_euler(seq='XYZ', angles=grasp_orientation, degrees=False)  # caps because instrinsic rotation
    default_orientation_rotation = R.from_euler(seq='XYZ', angles=default_orientation, degrees=False)
    
    # apply the orientation onto each other
    rotation_orientation = np.matmul(default_orientation_rotation.as_matrix(), grasp_orientation_rotation.as_matrix())
    
    # apply rotation noise
    desired_noise_rotation = R.from_rotvec(rotvec_noise)
    noisy_ori = np.matmul(desired_noise_rotation.as_matrix(), rotation_orientation)
    noisy_mujoco_euler = R.from_matrix(noisy_ori).as_euler(seq='XYZ')
    
    # get new postition
    noisy_mujoco_pos = rotate_around_origin(grasp_translation + local_rotation_pos_shift, rotvec_noise) + post_rotation_pos_shift
    
    return noisy_mujoco_euler, noisy_mujoco_pos
    
def stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr):
    return 'euler: ' +  ' '.join(noisy_mujoco_euler_arr.astype(str)) + '\n' + 'pos: ' + ' '.join(noisy_mujoco_pos_arr.astype(str))

In [16]:
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo([0, 0, np.pi/4], local_rotation_pos_shift=[0, 1, 0])
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: -1.5696701508795226 -0.7853978463293334 -1.5692036734575932
pos: -0.8343860018001261 0.834386001800126 0.0654


In [23]:
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo([0, 0, np.pi/4], local_rotation_pos_shift=[0, 0, 0], grasp_translation=[0, 0, 1], grasp_orientation=[0,0,3.14])
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

ValueError: Expected input of shape (3,) or (P, 3), got (6,).

In [43]:
kwarg_dict = {
    'rotvec_noise': [0, np.pi/4, 0],
    'local_rotation_pos_shift': [0, 0, 0],
    'grasp_translation': np.array([0, 0, 0.8]),
    'grasp_orientation': np.array([0, 0, 3.14])
}
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo(**kwarg_dict)
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: 0.0 0.7853981633974483 3.14
pos: 0.5656854249492381 0.0 0.565685424949238


In [44]:
np.deg2rad(5)

0.08726646259971647

In [53]:
kwarg_dict = {
    'rotvec_noise': [0, 0, np.deg2rad(30)],
    'local_rotation_pos_shift': [-0.1, 0, 0]
}
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo(**kwarg_dict)
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: -1.5698768078808927 -0.5235985925389177 -1.5695402405915897
pos: -0.17660254037844386 0.10588457268119897 0.0654


In [10]:
print(type(noisy_mujoco_euler_arr[0]))

<class 'numpy.float64'>


In [20]:
R.from_euler(seq='XYZ', angles=[0, 0, 0], degrees=False).apply(normal_orientation_rotation)

ValueError: Expected input of shape (3,) or (P, 3), got (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1).

In [11]:
list(normal_orientation_rotation.apply(position_vector_at_ground))

[0.17999994292773025, 0.06540009340826342, -9.125899560187316e-05]

In [5]:
# Disable jedi autocompleter
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import gym
import os

from scipy.spatial.transform import Rotation as R
# os.path.dirname(os.path.realpath(__file__))


# env = gym.make('gym_kinova_gripper:kinovagripper-v0')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
kwarg_dict = {
    'rotvec_noise': [0, 0, 0],
    'local_rotation_pos_shift': [0, 0, 0],
    'post_rotation_pos_shift': [0,0.18,0.0654],
#     'post_rotation_pos_shift': [0,0,0],
    'grasp_translation': np.array([0, 0, 0]),
    'grasp_orientation': np.array([-1.57, 0, -1.57])
}
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo(**kwarg_dict)
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: -1.5700000000000003 0.0 -1.57
pos: 0.0 0.18 0.0654


In [2]:
def real_like_hov_noise(rotvec_noise, post_rotation_pos_shift=np.array([0,0,0]),
                                  default_orientation=np.array([0,0,0]),
                                  grasp_translation=np.array([0,0.18,0.0654]),
                                  grasp_orientation=np.array([-1.57, 0, -1.57])):
    """
    rotates the wrist position and orientation just like the real world input. Give roll pitch and yaw!
    
    rotvec_noise: rotation noise to be apply extrinsically, and to rotate the coordinates accordingly
    post_rotation_pos_shift: shifting the hand in global coordinates, after the inner shift...
    default_orientation: the default orientation. should stay at 0s
    grasp_translation: mujoco's euler value for whatever grasp position you're using
    grasp_orientation: mujoco's pos value for whatever grasp position you're using
    """
    # get rotation objects from starting grasp orientation, and the default orientation (should be 0 0 0)
    grasp_orientation_rotation = R.from_euler(seq='XYZ', angles=grasp_orientation, degrees=False)  # caps because instrinsic rotation
    default_orientation_rotation = R.from_euler(seq='XYZ', angles=default_orientation, degrees=False)
    
    # apply the orientation onto each other
    rotation_orientation = np.matmul(default_orientation_rotation.as_matrix(), grasp_orientation_rotation.as_matrix())
    
    # apply rotation noise
    desired_noise_rotation = R.from_rotvec(rotvec_noise)
    noisy_ori = np.matmul(desired_noise_rotation.as_matrix(), rotation_orientation)
    noisy_mujoco_euler = R.from_matrix(noisy_ori).as_euler(seq='XYZ')
    
    # get new postition
    noisy_mujoco_pos = grasp_translation + post_rotation_pos_shift
    
    return noisy_mujoco_euler, noisy_mujoco_pos
    

In [14]:

# example usage of rotating using settings similar to how real-life HOV is done.
kwarg_dict = {
    'rotvec_noise': [np.deg2rad(30), 0, -np.deg2rad(30)],  # extrinsic rotation here
    'post_rotation_pos_shift': [0,0,0],  # positional noise here
    'grasp_translation': np.array([0,0.18,0.0654]),  # the grasp position here (built to work with our previous positioning system)
    'grasp_orientation': np.array([-1.57, 0, -1.57])  # the grasp orientation here (built to work with our previous orientation system)
}
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = real_like_hov_noise(**kwarg_dict)
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: -0.9961482218033897 0.49716991355617246 -1.7199551997157296
pos: 0.0 0.18 0.0654


In [15]:
# example usage of rotating around sphere

kwarg_dict = {
    'rotvec_noise': [0, 0, 0],  # extrinsic rotation of the wrist here
    'local_rotation_pos_shift': [0, 0, 0],  # positional noise wrt wrist's coordinate system here
    'post_rotation_pos_shift': [0,0,0],  # positional noise wrt global coordinate system here
    'grasp_translation': np.array([0,0.18,0.0654]),  # the grasp position here (built to work with our previous positioning system)
    'grasp_orientation': np.array([-1.57, 0, -1.57])  # the grasp orientation here (built to work with our previous orientation system)
}
noisy_mujoco_euler_arr, noisy_mujoco_pos_arr = rotate_around_sphere_mucrappo(**kwarg_dict)
print(stringify_mucrappo(noisy_mujoco_euler_arr, noisy_mujoco_pos_arr))

euler: -1.5700000000000003 0.0 -1.57
pos: 0.0 0.18 0.0654


In [19]:

cv2.imwrite('file name with extension(like .jpg)', numpy_array)
